### Meta Labeling

베팅(매도 또는 매수)의 방향을 설정하는 모델을 이미 갖고 있는 상황을 가정해 보자. 이 경우에는 베팅의 크기만 알면 되는데, 베팅이 없을 가능성도 있다. 이는 현업에서 항상 겪는 상황이기도 하다. 우리는 대개 매수할 것인지 매도할 것인지를 알고 있으므로, 남은 문제는 얼마를 집행할 것인지를 결정하는 것이다. 단지 머신러닝 알고리즘이 매수 매도 등 베팅의 방향을 학습한 후 적절한 베팅 크기가 무엇인지 쉽게 알려 주기를 원하지 않는다. 아마도 이 문제에 관한 어떠한 책이나 논문이 나오지 않았다는 것에 이미 익숙해져 버렸을 수 있다

다행히 이 문제를 여기서 해결해 볼 수 있다. Lopez de Prado는 이 문제를 Meta Labeling이라 부르는데 그 이유는 1차 외생 모델을 어떻게 사용할지 학습하는 부수적인 머신러닝 모델을 구축하고자 하기 때문이다

아래의 과정은 Triple Barrier Method에서 구축한 방법과 동일한 과정이다.

In [2]:
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from FinancialMachineLearning.labeling.labeling import *
from FinancialMachineLearning.features.volatility import daily_volatility

start_date = '2000-01-01'
end_date = '2024-04-08'
ticker = 'AAPL'
data = yf.download(
    ticker, 
    start = start_date, 
    end = end_date
)

[*********************100%%**********************]  1 of 1 completed


In [3]:
vertical_barrier = add_vertical_barrier(
    data.index, 
    data['Close'], 
    num_days = 7 # expariation limit
)
vertical_barrier.head()

Date
2000-01-03   2000-01-10
2000-01-04   2000-01-11
2000-01-05   2000-01-12
2000-01-06   2000-01-13
2000-01-07   2000-01-14
Name: Date, dtype: datetime64[ns]

In [4]:
volatility = daily_volatility(
    data['Close'], 
    lookback = 60 # moving average span
)

In [5]:
triple_barrier_events = get_events(
    close = data['Close'],
    t_events = data.index[2:],
    pt_sl = [2, 1], # profit taking 2, stopping loss 1
    target = volatility, # dynamic threshold
    min_ret = 0.01, # minimum position return
    num_threads = 1, # number of multi-thread 
    vertical_barrier_times = vertical_barrier, # add vertical barrier
    side_prediction = None # betting side prediction (primary model)
)
triple_barrier_events.head()

,t1,trgt,pt,sl
Date,,,,
2000-01-07,2000-01-11,0.016797,2,1
2000-01-10,2000-01-12,0.026415,2,1
2000-01-11,2000-01-12,0.024043,2,1
2000-01-12,2000-01-13,0.031168,2,1
2000-01-13,2000-01-19,0.050703,2,1


In [6]:
labels = meta_labeling(
    triple_barrier_events, 
    data['Close']
)

In [7]:
labels.head()

,ret,trgt,bin
Date,,,
2000-01-07,-0.067839,0.016797,-1
2000-01-10,-0.108056,0.026415,-1
2000-01-11,-0.059973,0.024043,-1
2000-01-12,0.109677,0.031168,1
2000-01-13,0.101422,0.050703,0


In [8]:
triple_barrier_events['side'] = labels['bin']
triple_barrier_events.head()

,t1,trgt,pt,sl,side
Date,,,,,
2000-01-07,2000-01-11,0.016797,2,1,-1.0
2000-01-10,2000-01-12,0.026415,2,1,-1.0
2000-01-11,2000-01-12,0.024043,2,1,-1.0
2000-01-12,2000-01-13,0.031168,2,1,1.0
2000-01-13,2000-01-19,0.050703,2,1,0.0


In [9]:
meta_labels = meta_labeling(
    triple_barrier_events, # with side labels
    data['Close']
)
meta_labels.head()

,ret,trgt,bin,side
Date,,,,
2000-01-07,0.072776,0.016797,1,-1.0
2000-01-10,0.121147,0.026415,1,-1.0
2000-01-11,0.063799,0.024043,1,-1.0
2000-01-12,0.109677,0.031168,1,1.0
2000-01-13,0.000000,0.050703,0,0.0


여기서 생성된 `meta_labels['bin']`이 가능한 값은 {-1, 0, 1}과 반대로 {0, 1}이다. Machine Learning 알고리즘은 오직 베팅을 하느냐, 하지 않느냐만 결정하고자 학습된다. 예측 label이 1이라면 2차 모델의 확률을 사용해 베팅의 크기를 알아내는데 포지션의 side(매수 혹은 매도인지)는 이미 primary model에 의해 결정되어 있다

### How to use Meta Labeling

binary classification 문제는 1종 오류(false positive)와 2종 오류(false negative) 간의 trade-off이다. 대개 binary classifier의 참 양성률을 올리다 보면 거짓 양성률이 증가한다. 분류 모형의 ROC(Receiver Operating Curve)는 true positive rate가 증가해 발생하는 비용을 측정하는데, 높은 false positive rate를 수용하는 관점에서 측정한다

아래의 그림은 이른바 Confusion matrix를 시각화한 것이다. 관측값 집합 중에서 조건에 부합하는 아이템과 조건에 부합하지 않는 아이템이 있다. binary classifier는 특정 아이템이 조건을 보이는지를 예측하는 것으로(타원), TP 영역은 True positive를 포함하고, TN 영역은 True negative를 포함한다. 이는 두 종류의 오류인 False positive와 False negative를 야기한다. **precision**은 TP영역과 타원 영역간의 비율이고, **recall**은 TP영역과 왼쪽 사각형 영역 간의 비율이다. 분류 문제에 있어서 이런 개념의 recall rate는 가설 검증에 있어서 검증력과 유사하다. **accuracy**는 TP와 TN 영역의 합을 전체 아이템으로 나눈 것이다. 

대개 FP영역을 감소시키면 FN영역이 증가하는데, 그 이유는 일반적으로 높은 정밀도는 더 적은 수의 예측을 의미하고 이는 **recall rate**을 낮추기 때문이다. 한편, 분류기 전체의 효율성을 극대화시키는 precision과 recall사이의 조합도 존재한다. **f1-score**는 분류기의 효율성을 측정하는데 precision과 recall 사이의 조화 평균을 사용해 측정한다.

